<a href="https://colab.research.google.com/github/aryajani/algo-trading/blob/main/smart_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages
!pip install smartapi-python
!pip install logzero
!pip install pyotp
!pip install websocket-client
!pip install ijson


In [ ]:
# Set time zone to IST
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
!date

In [ ]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from datetime import time, timedelta, datetime
import ijson
import json
import time
import requests

api_key = 'key'
username = 'user'
pwd = 'pin'
smartApi = SmartConnect(api_key)
try:
    token = "token"
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)


if data['status'] == False:
    logger.error(data)

else:
    # login api call
    # logger.info(f"You Credentials: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    # fetch the feedtoken
    feedToken = smartApi.getfeedToken()
    # fetch User Profile
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']


    def fetch_price(exchange, symbol, token):
        data = smartApi.ltpData(exchange, symbol, token)
        return data['data']['ltp']


    def find_nearest_expiry(url, strike_price):
        # Fetch the JSON data
        response = requests.get(url)
        data = json.loads(response.text)

        # Normalize the strike price
        normalized_strike = str(int(float(strike_price) * 100))

        # Separate lists for CE and PE options
        ce_options = []
        pe_options = []
        latest_ce = None
        latest_pe = None

        for entry in data:
            # Normalize the strike price from JSON
            json_strike = str(int(float(entry['strike'])))

            if (json_strike == normalized_strike and
                entry['exch_seg'] == "NFO" and
                entry['name'] == "NIFTY"):

                expiry_date = datetime.strptime(entry['expiry'], "%d%b%Y").date()

                if entry['symbol'][-2:] == "CE":
                    if latest_ce is None or expiry_date < datetime.strptime(latest_ce['expiry'], "%d%b%Y").date():
                        latest_ce = entry
                elif entry['symbol'][-2:] == "PE":
                    if latest_pe is None or expiry_date < datetime.strptime(latest_pe['expiry'], "%d%b%Y").date():
                        latest_pe = entry

        print(f"Latest CE: {latest_ce}")
        print(f"Latest PE: {latest_pe}")

        return {
            'CE': latest_ce,
            'PE': latest_pe
        }




    # Fetch Nifty price
    nifty_future = fetch_price("NFO", "NIFTY30JAN25FUT", "35006")
    print(f"Nifty future price: {nifty_future}")

    strike_price = round(nifty_future / 50) * 50
    print(f"Strike price: {strike_price}")

    json_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"

    # Find nearest CE and PE
    options = find_nearest_expiry(json_url, strike_price)

    price_cmp = fetch_price("NFO", options['CE']['symbol'], options['CE']['token'])
    price_pmp = fetch_price("NFO", options['PE']['symbol'], options['PE']['token'])

    print(f"CE price: {price_cmp}")
    print(f"PE price: {price_pmp}")

    sell_cmp_params = {
        'variety': 'NORMAL',
        'tradingsymbol': options['CE']['symbol'],
        'symboltoken': options['CE']['token'],
        'transactiontype': 'SELL',
        'exchange': 'NFO',
        'ordertype': 'LIMIT',
        'producttype': 'INTRADAY',
        'duration': 'DAY',
        'price': int(price_cmp*.975),
        'quantity': 25
    }

    try:
        sell_order_id_c = smartApi.placeOrder(sell_cmp_params)
        print(f"Sell Call order placed successfully, Order ID: {sell_order_id_c}")
    except Exception as e:
        print(f"An error occurred placing sell order: {e}")

    sell_pmp_params = {
        'variety': 'NORMAL',
        'tradingsymbol': options['PE']['symbol'],
        'symboltoken': options['PE']['token'],
        'transactiontype': 'SELL',
        'exchange': 'NFO',
        'ordertype': 'LIMIT',
        'producttype': 'INTRADAY',
        'duration': 'DAY',
        'price': int(price_pmp*.975),
        'quantity': 25
    }

    try:
        sell_order_id_p = smartApi.placeOrder(sell_pmp_params)
        print(f"Sell Put order placed successfully, Order ID: {sell_order_id_p}")
    except Exception as e:
        print(f"An error occurred placing sell order: {e}")

    # Wait a bit to ensure the sell order is placed before placing the stop loss
    time.sleep(5)

    OrderBook = smartApi.orderBook()['data']
    for i in OrderBook:
        if i['orderid'] == sell_order_id_c:
            price_sold_c = i['averageprice']
        if i['orderid'] == sell_order_id_p:
            price_sold_p = i['averageprice']


    stop_loss_c = price_sold_c * 1.30
    stop_loss_p = price_sold_p * 1.30

    stop_loss_cmp_params = {
        'variety': 'STOPLOSS',
        'tradingsymbol': options['CE']['symbol'],
        'symboltoken': options['CE']['token'],
        'transactiontype': 'BUY',
        'exchange': 'NFO',
        'ordertype': 'STOPLOSS_LIMIT',
        'producttype': 'INTRADAY',
        'duration': 'DAY',
        'quantity': 25,
        'triggerprice' : int(stop_loss_c),
        'price' : int(stop_loss_c*1.05)
    }

    try:
        stop_loss_order_id = smartApi.placeOrder(stop_loss_cmp_params)
        print(f"Stop Loss order CE placed successfully, Order ID: {stop_loss_order_id}")
    except Exception as e:
        print(f"An error occurred placing stop loss order: {e}")


    # Assuming the sell order was placed at 'sell_price'

    stop_loss_pmp_params = {
        'variety': 'STOPLOSS',
        'tradingsymbol': options['PE']['symbol'],
        'symboltoken': options['PE']['token'],
        'transactiontype': 'BUY',
        'exchange': 'NFO',
        'ordertype': 'STOPLOSS_LIMIT',
        'producttype': 'INTRADAY',
        'duration': 'DAY',
        'quantity': 25,
        'triggerprice' : int(stop_loss_p),
        'price' : int(stop_loss_p*1.05)
    }

    try:
        stop_loss_order_id = smartApi.placeOrder(stop_loss_pmp_params)
        print(f"Stop Loss order PE placed successfully, Order ID: {stop_loss_order_id}")
    except Exception as e:
        print(f"An error occurred placing stop loss order: {e}")

In [ ]:
modify_params = {
    'orderid' : "250127000323307",
    'variety': 'ROBO',
    'tradingsymbol': options['PE']['symbol'],
    'symboltoken': options['PE']['token'],
    'transactiontype': 'BUY',
    'exchange': 'NFO',
    'ordertype': 'STOPLOSS_LIMIT',
    'producttype': 'INTRADAY',
    'duration': 'DAY',
    'trailingStopLoss':0.50,
    'quantity': 25,
    'triggerprice' : int(stop_loss_p),
    'price' : int(stop_loss_p*1.05)
}

smartApi.modifyOrder(modify_params)

In [ ]:
def get_order_info(orderid):
    OrderBook = smartApi.orderBook()['data']
    for i in OrderBook:
        if i['orderid'] == orderid:
            return i


def trailing_stoploss(orderid_sl, trailing_stoploss, orderid): # trailing_stoploss =

    order_info_sl = get_order_info(orderid_sl)
    order_info = get_order_info(orderid)
    print(order_info_sl)

    new_stoploss = None

    symbol = order_info_sl['tradingsymbol']
    exchange = order_info_sl['exchange']
    token = order_info_sl['symboltoken']
    prev_price = order_info['averageprice']
    order_type = order_info_sl['transactiontype']
    prev_stoploss = order_info_sl['triggerprice']



    while True:

        curr_price = fetch_price(exchange, symbol, token)

        if order_type == 'BUY': # stoploss order for a serpe
            print("Current Price - ", curr_price)
            print("Prev Price - ", prev_price)
            if curr_price < prev_price:

                new_stoploss = int(prev_stoploss - (prev_price - curr_price) * trailing_stoploss)
                print("diff - ", (prev_price - curr_price))
                print("New Stoploss - ", new_stoploss)
        else:
            if curr_price > prev_price:

                new_stoploss = int(prev_stoploss + (curr_price - prev_price) * trailing_stoploss)

        if new_stoploss:
            new_params = order_info_sl
            new_params['triggerprice'] = new_stoploss
            new_params['price'] = int(new_stoploss * 1.05)
            prev_price = curr_price
            prev_stoploss = new_stoploss
            print(new_stoploss)
            smartApi.modifyOrder(new_params)

        time.sleep(15)

trailing_stoploss("250127000323307", 0.50, "250127000322889")

